In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
from tqdm import tqdm
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
plt.style.use('seaborn')
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.layers import Input, Conv1D, LSTM, Dense
from keras.models import Model

# 데이터 불러오기 및 전처리
df = pd.read_csv('/content/drive/MyDrive/csv_file/2023_smartFarm_AI_hackathon_dataset.csv')

encoder = LabelEncoder()
df['frmDist'] = encoder.fit_transform(df['frmDist'])

df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day"] = df["date"].dt.day

# 시계열 데이터 선택 및 NumPy 배열로 변환
normalized_data = df[['outtrn_cumsum', 'HeatingEnergyUsage_cumsum']].to_numpy()

time_steps = 7
n_series = normalized_data.shape[1]

input_sequences = []
target_values = []

for i in range(len(normalized_data) - time_steps):
    input_sequences.append(normalized_data[i:i+time_steps])
    target_values.append(normalized_data[i+time_steps])

input_sequences = np.array(input_sequences)
target_values = np.array(target_values)

# 데이터 분할
split = int(0.8 * len(input_sequences))
x_train = input_sequences[:split]
y_train = target_values[:split]
x_test = input_sequences[split:]
y_test = target_values[split:]

# 데이터 차원 변환 및 StandardScaler 적용
x_train = x_train.reshape(-1, n_series)
x_test = x_test.reshape(-1, n_series)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

x_train = x_train.reshape(-1, time_steps, n_series)
x_test = x_test.reshape(-1, time_steps, n_series)

# 모델 정의 (이하 모델 정의 및 학습 코드)
inputs = Input(shape=(time_steps, n_series))
conv_lstm_model = ConvLSTM(time_steps, n_series)
outputs = conv_lstm_model(inputs)

# 모델 학습 및 예측 (이하 학습 및 예측 코드)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense
from tensorflow.keras.models import Model

class ConvLSTM(Model):
    def __init__(self, time_steps, n_series):
        super(ConvLSTM, self).__init__()
        self.conv1d = Conv1D(32, kernel_size=2)
        self.lstm = LSTM(32)
        self.dense1 = Dense(32, activation='relu')
        self.dense2 = Dense(16, activation='relu')
        self.dense3 = Dense(n_series, activation='linear')

    def call(self, inputs):
        x = self.conv1d(inputs)
        x = self.lstm(x)
        x = self.dense1(x)
        x = self.dense2(x)
        outputs = self.dense3(x)
        return outputs

time_steps = 7
n_series = 2

inputs = Input(shape=(time_steps, n_series))
conv_lstm_model = ConvLSTM(time_steps, n_series)
outputs = conv_lstm_model(inputs)

conv_lstm_model.summary()

Model: "conv_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             multiple                  160       
                                                                 
 lstm (LSTM)                 multiple                  8320      
                                                                 
 dense (Dense)               multiple                  1056      
                                                                 
 dense_1 (Dense)             multiple                  528       
                                                                 
 dense_2 (Dense)             multiple                  34        
                                                                 
Total params: 10098 (39.45 KB)
Trainable params: 10098 (39.45 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=Adam(learning_rate=3e-4), loss='mean_squared_error')

history=model.fit(x_train, y_train,
          epochs=100,
          batch_size=64,
          validation_split=0.1,
          )

predicted_values1 = model.predict(x_test)

predicted_values_original = scaler.inverse_transform(predicted_values1)

r2 = r2_score(y_test, predicted_values1)

# 예측 결과와 실제 값 사이의 RMSE 값 계산
rmse = np.sqrt(mean_squared_error(y_test, predicted_values1))

print(f"R2 Score: {r2}")
print(f"RMSE: {rmse}")

Epoch 1/100
955/955 [==============================] - 19s 6ms/step - loss: 155698167808.0000 - val_loss: 219302248448.0000
Epoch 2/100
955/955 [==============================] - 6s 7ms/step - loss: 155374747648.0000 - val_loss: 218553008128.0000
Epoch 3/100
955/955 [==============================] - 5s 6ms/step - loss: 154547339264.0000 - val_loss: 217124814848.0000
Epoch 4/100
955/955 [==============================] - 6s 7ms/step - loss: 153301221376.0000 - val_loss: 215186931712.0000
Epoch 5/100
955/955 [==============================] - 5s 6ms/step - loss: 151748689920.0000 - val_loss: 212479459328.0000
Epoch 6/100
955/955 [==============================] - 5s 5ms/step - loss: 149256634368.0000 - val_loss: 208704864256.0000
Epoch 7/100
955/955 [==============================] - 7s 7ms/step - loss: 146270978048.0000 - val_loss: 204211961856.0000
Epoch 8/100
955/955 [==============================] - 5s 5ms/step - loss: 142773010432.0000 - val_loss: 198865190912.0000
Epoch 9/100
955

NameError: ignored

In [ ]:
initial_input = input_sequences[-1] ## LAST INPUT SEQUENCE
num_iterations = 50
forecasts = []
for _ in range(num_iterations):
    forecast = model.predict(np.expand_dims(initial_input, axis=0))[0]
    forecast_original = scaler.inverse_transform(forecast.reshape(1, -1))
    forecasts.append(forecast_original[0])
    initial_input = np.concatenate((initial_input[1:], forecast.reshape(1, -1)), axis=0)

In [ ]:
final_forecasts = np.array(forecasts)
r2_forecasts = r2_score(y_test[:len(final_forecasts)], final_forecasts)

# 예측 결과와 실제 값 사이의 RMSE 값 계산
rmse_forecasts = np.sqrt(mean_squared_error(y_test[:len(final_forecasts)], final_forecasts))

print(f"R2 Score for Forecasts: {r2_forecasts}")
print(f"RMSE for Forecasts: {rmse_forecasts}")